In [2]:
import sys
!{sys.executable} -m pip install geopy

     |████████████████████████████████| 111 kB 1.7 MB/s eta 0:00:01
You should consider upgrading via the '/Library/Frameworks/Python.framework/Versions/3.6/bin/python3 -m pip install --upgrade pip' command.


In [32]:
example_xls = xlrd.open_workbook("nbs-state-commodity-costs-data/selected_food_august_2020.xlsx")

In [33]:
example_xls.sheet_names()

['ABIA',
 'ABUJA',
 'ANAMBRA',
 'EBONYI',
 'ENUGU',
 'IMO',
 'AKWA IBOM',
 'BAYELSA',
 'CROSS RIVER',
 'DELTA',
 'RIVERS',
 'EDO',
 'ADAMAWA',
 'BAUCHI',
 'BORNO',
 'GOMBE',
 'TARABA',
 'YOBE',
 'BENUE',
 'KOGI',
 'KWARA',
 'NASSARAWA',
 'NIGER',
 'PLATEAU',
 'EKITI',
 'LAGOS',
 'ONDO',
 'OGUN ',
 'OSUN',
 'OYO',
 'JIGAWA ',
 'KADUNA ',
 'KANO ',
 'KATSINA',
 'KEBBI ',
 'ZAMFARA ',
 'SOKOTO',
 'NATIONAL']

In [8]:
STATES = [
    "Abia",
    "Adamawa",
    "Akwa Ibom",
    "Anambra",
    "Bauchi",
    "Bayelsa",
    "Benue",
    "Borno",
    "Cross River",
    "Delta",
    "Ebonyi",
    "Edo",
    "Ekiti",
    "Enugu",
    "Gombe",
    "Imo",
    "Jigawa",
    "Kaduna",
    "Kano",
    "Katsina",
    "Kebbi",
    "Kogi",
    "Kwara",
    "Lagos",
    "Nasarawa",
    "Niger",
    "Ogun",
    "Ondo",
    "Osun",
    "Oyo",
    "Plateau",
    "Rivers",
    "Sokoto",
    "Taraba",
    "Yobe",
    "Zamfara"
]

In [36]:
assert ([x.upper() for x in STATES]).sort() == ([x.strip() for x in example_xls.sheet_names()]).sort()

In [54]:
import xlrd
import csv
import os

def csv_from_excel(filename):
    wb = xlrd.open_workbook(filename)
    for sheet in wb.sheet_names():
        sh = wb.sheet_by_name(sheet)
        wb_name = os.path.splitext(filename)[0]
        csv_file = open(wb_name.strip() + "_" + sheet.strip() + '.csv', 'w')
        wr = csv.writer(csv_file, quoting=csv.QUOTE_ALL)

        start_writing = False
        for rownum in range(sh.nrows):
            row_values = sh.row_values(rownum)[:-2] # trim off aggregate MoM and YoY values (for now)
            # convert date values on header row of data
            if row_values[0] == "ItemLabels":
                # don't start writing to the sheet till "ItemLabels" row
                # b/c some of the data files have padding rows at the top that aren't needed
                start_writing = True 
                # print(row_values[1:])
                date_values = [xlrd.xldate.xldate_as_datetime(val, wb.datemode).strftime("%b %Y") for val in row_values[1:] if isinstance(val, float)]
                # print(date_values)
                row_values[1:len(date_values)+1] = date_values
            if start_writing: wr.writerow(row_values)

        csv_file.close()

In [7]:
import xlrd
import csv
import os

def joint_csv_from_excel(filename):
    wb = xlrd.open_workbook(filename)
    wb_name = os.path.splitext(filename)[0]
    csv_file = open(wb_name.strip() + "___JOINED.csv", 'w')
    wr = csv.writer(csv_file, quoting=csv.QUOTE_ALL)
    for sheet in wb.sheet_names():
        sh = wb.sheet_by_name(sheet)
                
        start_writing = False
        for rownum in range(sh.nrows):
            row_values = sh.row_values(rownum)[:-2] # trim off aggregate MoM and YoY values (for now)
            
            # convert date values on header row of data
            if row_values[0] == "ItemLabels":
                if rownum > 0: continue # only acknowledge header row on first sheet
                # don't start writing to the sheet till "ItemLabels" row
                # b/c some of the data files have padding rows at the top that aren't needed
                start_writing = True 
                # print(row_values[1:])
                date_values = [xlrd.xldate.xldate_as_datetime(val, wb.datemode).strftime("%b %Y") for val in row_values[1:] if isinstance(val, float)]
                # print(date_values)
                row_values[1:len(date_values)+1] = date_values
            
            row_values.insert(0, sheet) # insert name of state (corresponding to sheet in source .xlsx) to start of row
            
            if start_writing: wr.writerow(row_values)

    csv_file.close()

In [55]:
csv_from_excel("nbs-state-commodity-costs-data/selected_food_august_2020.xlsx")

In [56]:
csv_from_excel("nbs-state-commodity-costs-data/transport_cost_august_2020.xlsx")

above commands ([In 55]) and ([In 16]) should have created a new CSV in the same directory corresponding to each data-sheet with statistics for individual states

In [8]:
joint_csv_from_excel("nbs-state-commodity-costs-data/selected_food_august_2020.xlsx")

above command ([In 8]) should have created a new CSV containing the conglomerated data for each state.

In [3]:
from geojson import Feature, FeatureCollection, Point
from geopy.geocoders import Nominatim

In [26]:
def geojson_features_from_csv(filename):
    features = []
    geocode_map = {}
    with open(filename, newline='') as csvfile:
        reader = csv.reader(csvfile, delimiter=',')
        headers = next(reader)
        headers[0] = "State"
        geolocator = Nominatim(user_agent="nbs_state_commodity_costs_ipynb")
        for row in reader:
            location_name = row[0]
            location, latitude, longitude = None, None, None 
            if location_name not in geocode_map:
                print("Looking up location: ", location_name)
                location = geolocator.geocode(location_name + ", Nigeria");
                print("\t (long, lat): (%f, %f)" % (location.longitude, location.latitude))
                geocode_map[location_name] = location
            else:
                location = geocode_map[location_name]
            latitude, longitude = map(float, (location.latitude, location.longitude))
            
            features.append(
                Feature(
                    geometry = Point((longitude, latitude)),
                    properties = { headers[i]: row[i] for i in range(1, len(headers)) }
                )
            )
        csvfile.close()

    collection = FeatureCollection(features)
    with open("GeoObs.json", "w") as f:
        f.write('%s' % collection)
        f.close()

In [27]:
geojson_features_from_csv("nbs-state-commodity-costs-data/selected_food_august_2020___JOINED.csv")

Looking up location:  ABIA
	 (long, lat): (7.515307, 5.454095)
Looking up location:  ABUJA
	 (long, lat): (7.489297, 9.064331)
Looking up location:  ANAMBRA
	 (long, lat): (6.953184, 6.218314)
Looking up location:  EBONYI
	 (long, lat): (8.034891, 6.199692)
Looking up location:  ENUGU
	 (long, lat): (7.414306, 6.553609)
Looking up location:  IMO
	 (long, lat): (7.066965, 5.585946)
Looking up location:  AKWA IBOM
	 (long, lat): (7.841227, 4.940864)
Looking up location:  BAYELSA
	 (long, lat): (6.028898, 4.762979)
Looking up location:  CROSS RIVER
	 (long, lat): (8.520477, 5.867197)
Looking up location:  DELTA
	 (long, lat): (6.178417, 5.527306)
Looking up location:  RIVERS
	 (long, lat): (6.860409, 4.841603)
Looking up location:  EDO
	 (long, lat): (5.972271, 6.607658)
Looking up location:  ADAMAWA
	 (long, lat): (12.388189, 9.512977)
Looking up location:  BAUCHI
	 (long, lat): (10.028775, 10.622828)
Looking up location:  BORNO
	 (long, lat): (13.308003, 12.187539)
Looking up location: 